# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [72]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from itertools import islice

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [73]:
f = glob.glob('../event_data/*')[0]
print(f)
with open(f, 'r', encoding = 'utf8', newline = '') as csvfile:
    csvreader = csv.reader(csvfile)
    for line in islice(csvreader,5):
        print(line)
# comment, artist0, firstname 2, 3, 4, 5, 6,7,8,12,13,16

../event_data/2018-11-27-events.csv
['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userId']
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']
['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88']
['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88']
["The B

In [51]:
full_data_rows_list = []

for file in glob.iglob('../event_data/*', recursive = True):    
# reading xsv file
    with open(file, 'r', encoding = 'utf8', newline = '') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader) #skip header
# extracting each data row one by one and append it 
        for line in csvreader:
#             print(line)
              full_data_rows_list.append(line)
    
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [82]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="../images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [57]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster([('127.0.0.1')])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [58]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [59]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [63]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

session.execute("drop table if exists music_library")
try:
    session.execute('''
        CREATE TABLE IF NOT EXISTS music_library(
            sessionId int,
            itemInSession int,
            artist text,
            song text,
            length decimal,        
            primary key (sessionId, itemInSession))''')
except Exception as e:
    print(e)

                    

In [87]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [91]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute('''select artist, song, length from music_library where sessionId = 338 and itemInSession = 4;''')
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [121]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

session.execute("drop table if exists music_library_2")
try:
    session.execute('''
        CREATE TABLE IF NOT EXISTS music_library_2(
            sessionId int,
            itemInSession int,
            userId int, 
            first_name text,
            last_name text,
            artist text,
            song text ,      
            primary key ((userId,sessionId), itemInSession))''')
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_2 (sessionId, itemInSession, userId, first_name, last_name, artist, song)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]),int(line[10]),line[1],line[4],line[0],line[9]))
        
rows = session.execute('''select artist, song, first_name, last_name from music_library_2 where userId = 10 and sessionId = 182''')
print(rows)
for row in rows:
    print(row.artist, row.song, row.first_name,row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [129]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("drop table if exists music_library_3")
try:
    session.execute('''
        CREATE TABLE IF NOT EXISTS music_library_3(
            sessionId int,
            itemInSession int,
            userId int, 
            first_name text,
            last_name text,
            artist text,
            song text ,      
            primary key (song, userId, itemInSession))''')
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_3 (sessionId, itemInSession, userId, first_name, last_name, artist, song)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]),int(line[10]),line[1],line[4],line[0],line[9]))
        
rows = session.execute('''select first_name, last_name from music_library_3 where song ='All Hands Against His Own' ''')
print(rows)
for row in rows:
    print(row.first_name, row.last_name)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [130]:
try:
    session.execute('''DROP TABLE IF EXISTS music_library''')
except Exception as e:
    print(e)
    
try:
    session.execute('''DROP TABLE IF EXISTS music_library_2''')
except Exception as e:
    print(e)
    
try:
    session.execute('''DROP TABLE IF EXISTS music_library_3''')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [131]:
session.shutdown()
cluster.shutdown()